# Contract definition.
Based on ideas from Peyton-Jones & Eber

In [1]:
from typing import TypeVar, Generic
from adt import adt, Case
from enum import Enum
from decimal import Decimal
import datetime as dt
import json

```haskell
european :: Date -> Contract -> Contract
european t u = cWhen (at t) (u `cOr` zero)
```

In [2]:
T=TypeVar('T')


@adt
class Observable(Generic[T]):
    def __repr__(self):
        return f"{self._key.name}({self._value})"
    CONST: Case[T]   # has a constant value at all points in time
    DATE: Case       # has whatever date is passed
    
CONST=Observable.CONST

In [3]:
CONST(42)

CONST(42)

In [106]:
class Asset(Enum):
    def __repr__(self):
        return self.value
    BTC='BTC'
    CAD='CAD'
    USD='USD'

BTC,CAD,USD=Asset

In [107]:
BTC

BTC

In [108]:
@adt 
class Contract:
    ZERO: Case
    ONE: Case[Asset]
    GIVE: Case["Contract"]
    AND: Case["Contract","Contract"]
    OR: Case["Contract","Contract"]
    CONDITION: Case[Observable[bool], "Contract","Contract"]
    SCALE: Case[Observable[Decimal],"Contract"]
    WHEN: Case[dt.date,"Contract"]
    
ZERO = Contract.ZERO
ONE = Contract.ONE
GIVE = Contract.GIVE
AND=Contract.AND
OR=Contract.OR
CONDITION=Contract.CONDITION
SCALE=Contract.SCALE
WHEN=Contract.WHEN


def repr_contract(self):
    name=self._key.name
    value=self._value
    if value is None:
        return f"{name}()"
    if isinstance(value,tuple):
        return f"{name}({','.join(repr(v) for v in value)})"
    return f"{name}({repr(value)})"
Contract.__repr__=repr_contract

In [109]:
z=ZERO()

In [110]:
z

ZERO()

In [111]:
def observable_to_json(observable):
    return observable.match(
        const = lambda x: ['CONST',x],
        date = lambda d: ['DATE',str(d)]
    )

def  __contract_to_json(contract):
    return contract.match(
        **{
            'zero':lambda: ['ZERO'],
            'one':lambda asset: ['ONE',asset.name],
            'give':lambda c: ['GIVE',to_json(c)],
            'and':lambda left, right: ['AND', to_json(left), to_json(right)],
            'or':lambda left, right: ['OR',to_json(left), to_json(right)],
            'condition': lambda obs, left,right: ['CONDITION',to_json(obs),to_json(left),to_json(right)],
            'scale':lambda o,c: ['SCALE',to_json(o),to_json(c)],
            'when':lambda d,c: ['WHEN',str(d), to_json(c)]
        },
    )

def  contract_to_json(contract):
    return contract.match(
        ZERO=lambda: ['ZERO'],
        ONE=lambda asset: ['ONE',asset.name],
        GIVE = lambda c: ['GIVE',to_json(c)],
        AND = lambda left, right: ['AND', to_json(left), to_json(right)],
        OR = lambda left, right: ['OR',to_json(left), to_json(right)],
        CONDITION= lambda obs, left,right: ['CONDITION',to_json(obs),to_json(left),to_json(right)],
        SCALE = lambda o,c: ['SCALE',to_json(o),to_json(c)],
        WHEN = lambda d,c: ['WHEN',str(d), to_json(c)]
        
    )

def to_json(thing):
    try:
        return {
            Contract:contract_to_json,
            Observable:observable_to_json
        }[type(thing)](thing)
    except KeyError:
        raise Exception(f"KeyError with {thing}")

In [112]:
def from_json(l):
    if isinstance(l, str):
        if hasattr(Asset, l):
            return getattr(Asset, l)
    
    if not isinstance(l,list):
        return l
    
    head, *tail = l
    assert head.upper()==head
    
    args=[from_json(el) for el in tail]
    
    if hasattr(Contract,head):
        return getattr(Contract,head)(*args)
    elif hasattr(Observable,head):
        return getattr(Observable,head)(*args)
    else:
        raise Exception(f"Unknown constructor: {head}")


In [113]:
def pprint(contract):
    print(json.dumps(to_json(contract),indent=2))

In [114]:
ZERO()

ZERO()

In [115]:
to_json(ZERO())

['ZERO']

In [116]:
ONE(BTC)

ONE(BTC)

In [117]:
to_json(ONE(BTC))

['ONE', 'BTC']

In [118]:
from_json(_)

ONE(BTC)

In [119]:
AND(ONE(BTC),GIVE(ONE(CAD)))

AND(ONE(BTC),GIVE(ONE(CAD)))

In [120]:
pprint(AND(ONE(BTC),GIVE(ONE(CAD))))

[
  "AND",
  [
    "ONE",
    "BTC"
  ],
  [
    "GIVE",
    [
      "ONE",
      "CAD"
    ]
  ]
]


# Examples

In [121]:
def spottrade(left_amount:Decimal, left:Asset, right_amount:Decimal, right:Asset):
    return AND(
        SCALE(CONST(left_amount),ONE(left)),
        GIVE(SCALE(CONST(right_amount),ONE(right)))
    )

## Buy a BTC with 8000 USD

In [122]:
t1=spottrade(1,BTC,48000,USD)

In [123]:
t1

AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(48000),ONE(USD))))

In [124]:
pprint(GIVE(t1))

[
  "GIVE",
  [
    "AND",
    [
      "SCALE",
      [
        "CONST",
        1
      ],
      [
        "ONE",
        "BTC"
      ]
    ],
    [
      "GIVE",
      [
        "SCALE",
        [
          "CONST",
          48000
        ],
        [
          "ONE",
          "USD"
        ]
      ]
    ]
  ]
]


# Serialization
Various different ways of representing, formatting, and serializing the same tree structure

In [125]:
j=to_json(t1)

In [126]:
j

['AND',
 ['SCALE', ['CONST', 1], ['ONE', 'BTC']],
 ['GIVE', ['SCALE', ['CONST', 48000], ['ONE', 'USD']]]]

In [127]:
t2=from_json(j)

In [128]:
t2==t1

True

In [129]:
t1

AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(48000),ONE(USD))))

In [130]:
t2

AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(48000),ONE(USD))))

In [131]:
pprint(t1)

[
  "AND",
  [
    "SCALE",
    [
      "CONST",
      1
    ],
    [
      "ONE",
      "BTC"
    ]
  ],
  [
    "GIVE",
    [
      "SCALE",
      [
        "CONST",
        48000
      ],
      [
        "ONE",
        "USD"
      ]
    ]
  ]
]


## calls and puts

In [132]:

option=WHEN(
    dt.date(2020,7,1),                            # on the 1st of JULY, acquire the right
    OR(                                           # to EITHER
        ZERO(),                                   # do nothing, OR
        AND(                                      # trade  
            GIVE(                                 # $10,000USD
                SCALE(CONST(10000),ONE(USD))      # for
            ),                                    # 1 BTC
            SCALE(CONST(1),ONE(BTC))
        )
    )
)

In [133]:
option

WHEN(datetime.date(2020, 7, 1),OR(ZERO(),AND(GIVE(SCALE(CONST(10000),ONE(USD))),SCALE(CONST(1),ONE(BTC)))))

In [134]:
pprint(option)

[
  "WHEN",
  "2020-07-01",
  [
    "OR",
    [
      "ZERO"
    ],
    [
      "AND",
      [
        "GIVE",
        [
          "SCALE",
          [
            "CONST",
            10000
          ],
          [
            "ONE",
            "USD"
          ]
        ]
      ],
      [
        "SCALE",
        [
          "CONST",
          1
        ],
        [
          "ONE",
          "BTC"
        ]
      ]
    ]
  ]
]


### Utility functions for constructing options

In [135]:
def call(t, base, k, quote):
    'at time t acquire the right to purchase 1 unit of base at price k'
    return WHEN(
        t,
        OR(
            ZERO(),
            spottrade(1,base, k, quote)
        )
    )

def put(t,base,k,quote):
    'at time t acquire the right to sell 1 unit of base at price k'
    return WHEN(
        t,
        OR(
            ZERO(),
            spottrade(k,quote, 1, base)
        )
    )

def forward(t, contract):
    return WHEN(t,contract)

In [136]:
f= forward(dt.date(2021,12,31),spottrade(1,BTC,40427,USD))

f

WHEN(datetime.date(2021, 12, 31),AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(40427),ONE(USD)))))

In [137]:
pprint(f)

[
  "WHEN",
  "2021-12-31",
  [
    "AND",
    [
      "SCALE",
      [
        "CONST",
        1
      ],
      [
        "ONE",
        "BTC"
      ]
    ],
    [
      "GIVE",
      [
        "SCALE",
        [
          "CONST",
          40427
        ],
        [
          "ONE",
          "USD"
        ]
      ]
    ]
  ]
]


In [138]:
print(json.dumps(to_json(f) ))

["WHEN", "2021-12-31", ["AND", ["SCALE", ["CONST", 1], ["ONE", "BTC"]], ["GIVE", ["SCALE", ["CONST", 40427], ["ONE", "USD"]]]]]


On July 1st I acquire a BTC for $10,000

In [141]:
forward(dt.date(2020,7,1),spottrade(1,BTC,10000,USD))

WHEN(datetime.date(2020, 7, 1),AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(10000),ONE(USD)))))

On July 1st, *either* obtain a BTC for 8,000 USD or do nothing.

In [142]:
call(dt.date(2020,7,1),BTC,10000,USD)

WHEN(datetime.date(2020, 7, 1),OR(ZERO(),AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(10000),ONE(USD))))))

In [143]:
pprint(call(dt.date(2020,7,1),BTC,10000,USD))

[
  "WHEN",
  "2020-07-01",
  [
    "OR",
    [
      "ZERO"
    ],
    [
      "AND",
      [
        "SCALE",
        [
          "CONST",
          1
        ],
        [
          "ONE",
          "BTC"
        ]
      ],
      [
        "GIVE",
        [
          "SCALE",
          [
            "CONST",
            10000
          ],
          [
            "ONE",
            "USD"
          ]
        ]
      ]
    ]
  ]
]


# PPN

* maturity: july 1st
* floor: 8k
* ceiling: 15k

In [144]:
t = dt.date(2020,7,1)

long_put=put(t,BTC,8000,USD)

long_call = call(t, BTC, 15000, USD)
short_call=GIVE(long_call)
 

In [145]:
long_put

WHEN(datetime.date(2020, 7, 1),OR(ZERO(),AND(SCALE(CONST(8000),ONE(USD)),GIVE(SCALE(CONST(1),ONE(BTC))))))

In [146]:
short_call

GIVE(WHEN(datetime.date(2020, 7, 1),OR(ZERO(),AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(15000),ONE(USD)))))))

In [147]:
ppn = AND(
    WHEN(t,ONE(BTC)),
    AND(
        short_call,
        long_put
    )
)

In [148]:
ppn

AND(WHEN(datetime.date(2020, 7, 1),ONE(BTC)),AND(GIVE(WHEN(datetime.date(2020, 7, 1),OR(ZERO(),AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(15000),ONE(USD))))))),WHEN(datetime.date(2020, 7, 1),OR(ZERO(),AND(SCALE(CONST(8000),ONE(USD)),GIVE(SCALE(CONST(1),ONE(BTC))))))))


## Complete transaction

And then let's say we sold this PPN for, say 10000, the complete trade would be:

In [149]:
fulltrade=AND(
    SCALE(CONST(10000),ONE(USD)),
    GIVE(ppn)
)

In [150]:
fulltrade

AND(SCALE(CONST(10000),ONE(USD)),GIVE(AND(WHEN(datetime.date(2020, 7, 1),ONE(BTC)),AND(GIVE(WHEN(datetime.date(2020, 7, 1),OR(ZERO(),AND(SCALE(CONST(1),ONE(BTC)),GIVE(SCALE(CONST(15000),ONE(USD))))))),WHEN(datetime.date(2020, 7, 1),OR(ZERO(),AND(SCALE(CONST(8000),ONE(USD)),GIVE(SCALE(CONST(1),ONE(BTC))))))))))

In [151]:
pprint(fulltrade)

[
  "AND",
  [
    "SCALE",
    [
      "CONST",
      10000
    ],
    [
      "ONE",
      "USD"
    ]
  ],
  [
    "GIVE",
    [
      "AND",
      [
        "WHEN",
        "2020-07-01",
        [
          "ONE",
          "BTC"
        ]
      ],
      [
        "AND",
        [
          "GIVE",
          [
            "WHEN",
            "2020-07-01",
            [
              "OR",
              [
                "ZERO"
              ],
              [
                "AND",
                [
                  "SCALE",
                  [
                    "CONST",
                    1
                  ],
                  [
                    "ONE",
                    "BTC"
                  ]
                ],
                [
                  "GIVE",
                  [
                    "SCALE",
                    [
                      "CONST",
                      15000
                    ],
                    [
                      "ONE",
            

### Outstanding questions:

* can we use this to express a cash-settled future?
* can we tag or identify sub-trees in the contract expression?
* can we compare two trees for equivalence?
* how do we capture exchange features of futures, such as margin obligations?
* settlement
* option exercise, especially if we get into American options
* quanto: imagine we have a BTC/CAD trade but agree to settle in USD, settlement currency is unrelated
  - (e.g. USD speculation on the Nikkei)
* deribit options: BTC/USD trade but BTC settled
* Observable reference rates, e.g. Bitmex.

* Can we use pattern-matching to capture/identify/process sub-elements of a contract?

I feel this might be the most important question. A language with sum types yet without the ability to pattern match on sum-type constructors is maybe not very useful.

In Haskell, pattern matching in fact _only_ works on constructors

Now we clearly do have pattern matching, because that's what we've been using for serialization etc, e.g.:

But this is only matching on the constructor function name, which isn't really powerful enough. Can we do more than that?

## Futures